In [24]:
import sklearn
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import re
import pickle
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [25]:
conn = sqlite3.connect('../db.sqlite3')

In [30]:
df_raw = pd.read_sql_query("SELECT * FROM scraper_auto", conn)

In [31]:
target_names = ["bouwjaar", "kilometer_stand", "vermogen", "prijs", "is_handgeschakeld", "is_benzine", "upload_datum", "apk"]
df = df_raw[target_names]
df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,1130.000000,1.129000e+03,1130.000000,1.119000e+03,1128.000000,1130.000000
mean,2009.821239,1.676765e+05,83.763717,1.799459e+07,0.975177,0.574336
std,2.889137,9.856303e+04,13.537466,2.461839e+08,0.155653,0.494662
min,2005.000000,7.183000e+03,51.000000,1.000000e+00,0.000000,0.000000
25%,2007.000000,1.181140e+05,77.000000,2.250000e+03,1.000000,0.000000
50%,2010.000000,1.699530e+05,84.000000,2.950000e+03,1.000000,1.000000
75%,2011.000000,2.106970e+05,86.000000,4.350000e+03,1.000000,1.000000
max,2018.000000,2.625966e+06,180.000000,3.945143e+09,1.000000,1.000000


In [32]:
MAX_PRICE = 30000
MIN_PRICE = 1000
df = df[(df['prijs'] > MIN_PRICE) & (df['prijs'] < MAX_PRICE)]

MIN_BOUWJAAR = 2000
df = df[df['bouwjaar'] > MIN_BOUWJAAR]

MAX_VERMOGEN = 500
df = df[df['vermogen'] < MAX_VERMOGEN]

df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,1086.000000,1.085000e+03,1086.000000,1086.000000,1084.000000,1086.000000
mean,2009.919890,1.653619e+05,83.731123,3747.595764,0.975092,0.574586
std,2.882656,9.885061e+04,13.266203,2293.183763,0.155916,0.494633
min,2005.000000,7.183000e+03,51.000000,1100.000000,0.000000,0.000000
25%,2007.000000,1.172710e+05,78.000000,2350.000000,1.000000,0.000000
50%,2010.000000,1.690000e+05,84.000000,2950.000000,1.000000,1.000000
75%,2011.000000,2.082870e+05,86.000000,4387.500000,1.000000,1.000000
max,2018.000000,2.625966e+06,180.000000,13900.000000,1.000000,1.000000


In [33]:
df['upload_datum'] = pd.to_datetime(df['upload_datum'])
df['apk'] = pd.to_datetime(df['apk'])

average_upload_datum = df['upload_datum'].mean().toordinal()
average_apk =df['apk'].mean().toordinal()

def upload_datum_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_upload_datum)
    else:
        return abs(datetime.now().toordinal() - date.toordinal())
    
def apk_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_apk)
    else:  
        return abs(datetime.now().toordinal() - date.toordinal())

df['upload_datum'] = df['upload_datum'].apply(upload_datum_to_ordinal).astype(int)
df['apk'] = df['apk'].apply(apk_to_ordinal).astype(int)

# def parse_titel(titel):
#     cleaned = re.sub('[^a-zA-Z]+', ' ', titel.lower())
#     stripped = line = re.sub(r'\b\w{1,3}\b', '', cleaned)
    
#     return stripped

# df['titel'] = df['titel'].apply(parse_titel)

In [34]:
df = df.dropna()
#df.isna().sum()

In [35]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [36]:
df.to_pickle("./data.pkl")